In [1]:
import pandas as pd
import random
from collections import Counter

In [21]:
df = pd.read_csv('linke_scraper3.csv', index_col=0)

In [22]:
sample_dupli = df.sample(15)

In [26]:
test = pd.concat([df, sample_dupli])
test = test[['info', 'exp', 'apt1', 'apt2']]
test.reset_index(inplace=True, drop=True)
test.to_csv('test_mdevz.csv')

# Clean positions (test)

In [ ]:
exp = df.exp.to_list()

In [6]:
all_positions = []
for experience in exp:
    
    f = experience

    f = f.split('\r\n')
    
    if len(f) < 3:
        all_positions.append(['nada'])
        continue

    posicion = []
    positions = []
    nombre = 0
    for e, text in enumerate(f):

        if f[e] != 'Nombre de la empresa':
            posicion.append(text)
        else:
            nombre += 1
            if nombre == 2:
                if f[e+2] == 'Duración total':
                    positions.append(posicion)
                else:
                    posicion.pop(-1)
                    positions.append(posicion)
                all_positions.append(posicion)
                break
            else:
                continue
        if (e+1) == len(f):
            all_positions.append(posicion)
            break

In [7]:
rol, company, fecha, duracion = [], [], [], []
ubicacion = []

for position in all_positions:
    if 'Duración total' in position:
        company.append(position[0])
        e = 1
        cargo = 0
        while True:
            if 'Cargo' == position[e]:
                cargo += 1
                if cargo > 1: break
                rol.append(position[e+1])
            elif 'Fechas de empleo' == position[e]:
                fecha.append(position[e+1])
            elif 'Duración del empleo' == position[e]:
                duracion.append(position[e+1])
            elif 'Ubicación' == position[e]:
                ubicacion.append(position[e+1])
            e += 1
        continue
    if len(position) > 1:
        rol.append(position[0])
        company.append(position[1])
        fecha.append(position[3])
        duracion.append(position[5])
        if 'Ubicación' in position:
            ubicacion.append(position[7])
        else:
            ubicacion.append('None')
    else:
        rol.append('nada')
        company.append('nada')
        fecha.append('nada')
        duracion.append('nada')
        ubicacion.append('nada')

In [8]:
len(rol)

390

# Descriptions

In [9]:
descriptions = []

for position in all_positions:
    descri = []
    
    if 'Duración total' in position:
        cargo = 0
        e=0
        while True:
            if 'Cargo' == position[e]:
                cargo += 1
                if cargo == 2:
                    i = e-1
                    while (position[i] != 'Ubicación') & (position[i] != 'Duración del empleo'):
                        descri.insert(0, position[i])
                        i -= 1
                    break
            e += 1
        
        descri.pop(0)
        descriptions.append(descri)
        continue
    
    e = -1
    if len(position) < 3:
        descriptions.append([])
        continue
    while (position[e] != 'Ubicación') & (position[e] != 'Duración del empleo'):
        descri.insert(0, position[e])
        e -= 1
    descri.pop(0)
    descriptions.append(descri)

In [10]:
len(descriptions)

390

# Clean aptitudes

In [11]:
apt1 = df.apt1.to_list()

In [12]:
aptitudes = []
for apt in apt1:
    apti = []
    apt = apt.split('\r\n')
    if len(apt) == 1: 
        aptitudes.append(['nada'])
        continue
    if len(apt) > 1 and len(apt) < 4:
        aptitudes.append(apt)
        continue
    for e, text in enumerate(apt):
        if (('validado esta aptitud' in text) and (e != (len(apt) -1))):
            if ('aptitudes de LinkedIn' in apt[e+1]):
                if (e+1 != (len(apt) -1)):
                    apti.append(apt[e+2])
                else:
                    continue
            else:
                apti.append(apt[e+1])
        elif ('validaciones de' in text or 'validación de' in text or 'Aún no hay' in text):
            apti.append(apt[e-1])
    aptitudes.append(list(set(apti)))

# Aptitudes 2

In [13]:
apt2 = df.apt2.to_list()

In [14]:
aptitudes2 = []

remove = ['Conocimiento del sector', 'Herramientas y tecnologías', 'Otras aptitudes', 'Aptitudes interpersonales']

for apt in apt2:
    apt = apt.split('\r\n')
    apti2 = []
    for e, text in enumerate(apt):
        if ('validaciones' not in text) and ('validación' not in text):
            if text not in remove:
                try:
                    int(text)
                except:
                    apti2.append(text)
    aptitudes2.append(apti2)

In [15]:
ap = []
for e, i in zip(aptitudes, aptitudes2):
    ap.append(e + i)

# Info

In [16]:
info = df['info'].to_list()

In [28]:
info

['Dor Bernsohn\r\nContacto de 2º grado\r\n2º\r\nData Scientist @Darrow.ai\r\nDarrow\r\nThe Hebrew University of Jerusalem\r\nDistrito de Tel Aviv, Israel Información de contacto',
 'Erez Keidan\r\nContacto de 2º grado\r\n2º\r\nData Scientist at Isracard\r\nDistrito de Tel Aviv, Israel Información de contacto',
 'Hadar Lackritz\r\nContacto de 2º grado\r\n2º\r\nSenior Data Scientist at Intuit\r\nIntuit\r\nBen-Gurion University of the Negev\r\nIsrael Información de contacto',
 'Or Wolkomir\r\nContacto de 2º grado\r\n2º\r\nData Scientist at Lili\r\nLili\r\nTel Aviv University\r\nIsrael Información de contacto',
 'Dana Koslawsky\r\nContacto de 2º grado\r\n2º\r\nData Scientist at Anodot\r\nAnodot\r\nBen-Gurion University of the Negev\r\nIsrael Información de contacto',
 'Orly levy\r\nContacto de 2º grado\r\n2º\r\nData Scientist at ironSource\r\nironSource\r\nThe Hebrew University\r\nIsrael Información de contacto',
 'Dvir Ben Or\r\nContacto de 2º grado\r\n2º\r\nData Scientist at Playtika\r\n

In [17]:
name, location = [], []
for inf in info:
    inf = inf.split('\r\n')
    name.append(inf[0])
    if inf[-1] == 'Israel Información de contacto':
        location.append(inf[-2])
    else:
        location.append(inf[-1])

# New df

In [18]:
df_new = pd.DataFrame({'name':name, 'location':location,'actual_role':rol,'company':company,'fecha':fecha,
                       'duration':duracion, 'description': descriptions, 'aptitudes':ap})

In [19]:
df_new = df_new[df_new.name != 'nada']

In [20]:
df_new

,name,location,actual_role,company,fecha,duration,description,aptitudes
0,Dor Bernsohn,"Distrito de Tel Aviv, Israel Información de co...",Data Scientist,Darrow,nov 2021 – actualidad,2 meses,[],"[Deep Learning, Machine Learning, Natural Lang..."
1,Erez Keidan,"Distrito de Tel Aviv, Israel Información de co...",Data Scientist,Isracard,ago 2021 – actualidad,5 meses,[],"[java, Object-Oriented Programming (OOP), Prog..."
2,Hadar Lackritz,Ben-Gurion University of the Negev,Senior Data Scientist,Intuit Jornada completa,ene 2021 – actualidad,1 año,[],"[Data Science, Statistical Data Analysis, Mach..."
3,Or Wolkomir,Tel Aviv University,Data Scientist,Lili,ago 2021 – actualidad,5 meses,[],"[Python, Microsoft Office, HTML, Extract, Tran..."
4,Dana Koslawsky,Ben-Gurion University of the Negev,Data Scientist,Anodot,sept 2019 – actualidad,2 años y 4 meses,[Development ML algorithms for time series ana...,"[nada, nada]"
...,...,...,...,...,...,...,...,...
384,Avi Aminov,Technion-Machon Technologi Le' Israel,Chief Data Scientist,Authomize Jornada completa,dic 2019 – actualidad,2 años y 1 mes,[],"[Physics, Algorithms, Security, Object Oriente..."
385,Daniel Peleg,Technion - Israel Institute of Technology,Data Scientist,IIT Image Information Technologies,feb 2020 – actualidad,1 año y 11 meses,[],"[Team Leadership, Data Science, Research and D..."
386,Yotam Luz,Ben Gurion University,Lead Data Scientist,Vonage Jornada completa,2019 – actualidad,2 años,[],"[Software Development, R&D, Algorithms, Object..."
387,Yoav Gal,"Distrito de Tel Aviv, Israel Información de co...","Data Science Team Lead, Waze",Google,nov 2021 – actualidad,2 meses,[],"[Statistics, Python, Machine Learning, SQL, Py..."


In [208]:
df_new.to_csv('linkedin_parseado.csv')

# Some exploration

In [151]:
apti = df_new.apt2.to_list()

list_apt = []
for e in apti:
    list_apt += e

count_apt = Counter(list_apt)

count_apt = pd.DataFrame.from_dict(count_apt, orient='index').reset_index()

count_apt.columns = ['aptitudes', 'count']

In [154]:
count_apt.sort_values('count', ascending=False).head(15)

,aptitudes,count
41,SQL,64
12,R,46
33,Machine Learning,44
2,Data Analysis,43
40,Python (Programming Language),42
34,Deep Learning,39
6,Algorithms,37
37,Java,37
17,Microsoft Excel,36
11,Matlab,36
